In [0]:
import json
from pathlib import Path
from layer_03_gold import edsm_gold_transform, edsm_gold_write
from functions import create_table_if_not_exists
from pyspark.sql.functions import col

settings = json.loads(Path("../layer_03_gold/systemsPopulated.json").read_text())


# settings["src_table_name"] = "edsm.silver.powerplay_rescue"
# settings["dst_table_name"] = "edsm.gold.powerplay_rescue"

history = spark.sql(f"DESCRIBE HISTORY {settings['src_table_name']}")
max_version = history.agg({"version":"max"}).first()[0]
print(f"Max version: {max_version}")

spark.sql(f"DROP TABLE IF EXISTS {settings['dst_table_name']}")
dbutils.fs.rm(settings["writeStreamOptions"]["checkpointLocation"], recurse=True)

for version in range(0, max_version+1):
    if version == 0:
        df = spark.read.format("delta").option("versionAsOf", version).table(settings["src_table_name"])
        df = edsm_gold_transform(spark, settings, df)
        create_table_if_not_exists(spark, df, settings["dst_table_name"])
        print(f"Current version: {version}")
        continue

    df = spark.read.format("delta").option("versionAsOf", version).table(settings["src_table_name"])
    df = edsm_gold_transform(spark, settings, df)
    edsm_gold_write(spark, settings, df)
    print(f"Current version: {version}")